# フロントエンドのセットアップ

## 前提

このnotebookでは、Open OnDemandのフロントエンド部分の設定を行います。Slurmとの結合のための設定については、notebookを分けて説明します。
* [010-インストール](010-インストール.ipynb)のnotebookに従ってインストールが完了していること。
* mdxのDNAT設定で割り当てられるグローバルIPアドレスに空きがあること。

## 準備

### group_varsの読み込み

group_varsの読み込みに先立ち、ユニットグループ名をチェックします。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

`ugroup_name`にユニットグループ名を指定します。

In [ ]:
ugroup_name = 

group_varsを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### ansibleの設定確認

ansibleでSlurmクラスタにアクセスできることを確認します。

In [ ]:
!ansible -m ping {ugroup_name}

ansibleで指定するノード集合を設定します。

In [ ]:
target_master = f'{ugroup_name}_master'
target_compute = f'{ugroup_name}_compute'

## DNATエントリの追加

mdxのユーザポータル上で、マスターノードへのDNATエントリを追加して外部からアクセスできるようにします。この作業はnotebook上からは実行できないので、別途Webブラウザからmdxのユーザポータル上で作業してください。

## ユーザの追加

Open OnDemandでジョブを実行するユーザを各ノードに追加します。

追加するユーザ名を、ooduser変数に設定します。

In [ ]:
ooduser = 'ooduser'

Open OnDemandからジョブを実行するユーザをSlurmクラスタ全体に追加します。  
OpenHPC v2テンプレートで作成したSlurm環境では、/homeはNFSで共有されているので、計算ノードではユーザの作成のみでホームディレクトリを作成しないようにします。

In [ ]:
!ansible {target_master} -b -m user -a 'name={ooduser}'
!ansible {target_compute} -b -m user -a 'name={ooduser} create_home=false'

Open OnDemandからのシェルアクセス有効化のため、作成したユーザのSSH鍵の作成と`authorized_keys`を配置します。

In [ ]:
!ansible {target_master} -b -a \
    "su {ooduser} -c 'ssh-keygen -t rsa -b 2048 -N \"\" -f ~/.ssh/id_rsa'"

In [ ]:
!ansible {target_master} -b -m copy -a \
    'src=/home/{ooduser}/.ssh/id_rsa.pub \
    dest=/home/{ooduser}/.ssh/authorized_keys \
    owner={ooduser} group={ooduser} \
    remote_src=true'

テストのために、`ooduser`で指定したユーザでジョブが実行できることを確認します。

まず、Slurmクラスタの状態を確認します。`STATE`が`idle`となっている必要があります。

In [ ]:
!ansible {target_master} -a 'sinfo'

ジョブを実行します。

In [ ]:
!ansible {target_master} -b -a \
    'su - {ooduser} -c "srun -N {gvars['compute_nodes']} id"'

この設定では、Open OnDemandへのログインにbasic認証を使用します。`ooduser`に設定したユーザ名に対応する、basic認証のパスワードを入力します。ここで入力したパスワードは、Open OnDemandでのログインに使用しますので、覚えておいてください。

In [ ]:
from getpass import getpass
oodhtpasswd = getpass(f'password for {ooduser} basic auth:')

htpasswdファイルを作成します。

In [ ]:
htpasswd_file = '/etc/ood/config/htpasswd'
!ansible {target_master} -b -m file -a \
    'path={htpasswd_file} mode=0640 owner=apache group=apache state=touch'

`ooduser`のためのエントリを作成します。

In [ ]:
!ansible {target_master} -b -a 'htpasswd -b5 {htpasswd_file} {ooduser} {oodhtpasswd}'

## Open OnDemandサービス起動

### ユーザ認証関連の設定

Open OnDemand設定の、認証のためのセクションを記述します。

In [ ]:
import os
import yaml

ood_portal_file = 'ood_portal.yml'
oodauth = {
    'auth': [
        'AuthType basic',
        'AuthName "Open OnDemand"',
        'Require valid-user',
        'AuthUserFile "/etc/ood/config/htpasswd"',
    ]
}

if os.path.exists(ood_portal_file):
    with open(ood_portal_file) as f:
        ood_portal_conf = yaml.safe_load(f)
else:
    ood_portal_conf = {}

ood_portal_conf.update(oodauth)
ood_portal_conf

YAML形式のファイルに出力します。

In [ ]:
with open(ood_portal_file, 'w') as f:
    f.write('# /etc/ood/config/ood_portal.yml\n---\n')
    yaml.safe_dump(ood_portal_conf, f)

ファイルの内容を確認します。

In [ ]:
!cat {ood_portal_file}

### SSL (HTTPS)の設定

Open OnDemandにHTTPSでアクセスするための設定をします。HTTPでもアクセスできるため、必須ではありませんが、HTTPSでの使用を強くおすすめします。

HTTPの場合、Open OnDemandが使用しているRuby on RailsでのCSRF (Cross-Site Request Forgery) チェックが期待通りに動作しない(正当なアクセスをCSRFと判定する)ことにより、動作に支障をきたすコンポーネントがあるためです。

マスターノードに割り当てられたDNATの外部アドレスを別途DNS登録します。そのDNS上のホスト名を、以下の`ood_server_name`に設定します。

In [ ]:
ood_server_name = 

`ood_server_name`に設定したホストのSSL関連のファイルを、別途マスターノードに配置します。ここでは、以下のように配置するものとして設定します。

* 証明書: `/etc/pki/tls/certs/{ood_server_name}.crt`
* 秘密鍵: `/etc/pki/tls/private/{ood_server_name}.key`
* 中間証明書: `/etc/pki/tls/certs/{ood_server_name}-interm.crt`

これらのファイルがマスターノードに配置されていることを、以下のセルにより確認します。

In [ ]:
!ansible {target_master} -m file -b -a "path=/etc/pki/tls/certs/{ood_server_name}.crt state=file"
!ansible {target_master} -m file -b -a "path=/etc/pki/tls/private/{ood_server_name}.key state=file"
!ansible {target_master} -m file -b -a "path=/etc/pki/tls/certs/{ood_server_name}-interm.crt state=file"

`ood_portal.yml`の認証に関する部分の設定を生成します。

In [ ]:
if os.path.exists(ood_portal_file):
    with open(ood_portal_file) as f:
        ood_portal_conf = yaml.safe_load(f)

ood_portal_conf['ssl'] = [
    'SSLCertificateFile "/etc/pki/tls/certs/{}.crt"'.format(ood_server_name),
    'SSLCertificateKeyFile "/etc/pki/tls/private/{}.key"'.format(ood_server_name),
    'SSLCertificateChainFile "/etc/pki/tls/certs/{}-interm.crt"'.format(ood_server_name)
]

ood_portal_conf['servername'] = ood_server_name

ood_portal_conf

YAML形式のファイルに出力します。

In [ ]:
with open(ood_portal_file, 'w') as f:
    f.write('# /etc/ood/config/ood_portal.yml\n---\n')
    yaml.safe_dump(ood_portal_conf, f)

ファイルの内容を確認します。

In [ ]:
!cat {ood_portal_file}

### 設定ファイルの配置

作成したファイルをマスターノードに配置します。

In [ ]:
!ansible {target_master} -b -m copy -a \
    'src={ood_portal_file} dest=/etc/ood/config/{ood_portal_file} \
    backup=true'

配置した`ood_portal.yml`からApache httpdの設定ファイルを生成します。

In [ ]:
!ansible {target_master} -b -a \
    '/opt/ood/ood-portal-generator/sbin/update_ood_portal'

Open OnDemandのサービス(Apache httpd)を起動します。

In [ ]:
!ansible {target_master} -b -m systemd -a 'name=httpd state=started enabled=true'

Webブラウザから、`https://(ood_server_nameで指定したホスト名)/`にアクセスします。認証が求められるので、[ユーザの追加](#ユーザの追加)の節で追加したユーザ名と、basic認証のパスワードを入力します。

正しく設定できていれば、Open Ondemandのトップページが表示されます。ただし、ここまでの設定では、Slurmとの関連付けがなされていないので、まだジョブを投入することはできません。

Open OnDemandとSlurmとの関連付けの設定は、notebookを分けて説明します。